# Raisin Class Classification using a Dense Layer

In [4]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score

In [5]:
data = pd.read_excel("data/Raisin_dataset.xlsx")

In [6]:
le = LabelEncoder()
data['Class'] = le.fit_transform(data['Class'])
data['Class'].unique()

array([1, 0])

In [7]:
X = data.drop('Class', axis=1)
y = data['Class']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values)

In [42]:
class SimpleDenseNet(nn.Module):
    def __init__(self, input_dim):
        super(SimpleDenseNet, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [43]:
net = SimpleDenseNet(X_train.shape[1])
print(net)

SimpleDenseNet(
  (linear): Linear(in_features=7, out_features=1, bias=True)
)


In [46]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [47]:
num_epochs = 1000
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        outputs = net(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 31.2500
Epoch [200/1000], Loss: 31.2500
Epoch [300/1000], Loss: 31.2500
Epoch [400/1000], Loss: 31.2500
Epoch [500/1000], Loss: 31.2500
Epoch [600/1000], Loss: 31.2500
Epoch [700/1000], Loss: 31.2500
Epoch [800/1000], Loss: 31.2500
Epoch [900/1000], Loss: 31.2500
Epoch [1000/1000], Loss: 31.2500


In [48]:
net.eval()
with torch.no_grad():
    y_pred = net(X_test_tensor)
    y_pred_class = (y_pred.squeeze() > 0.5).float()
    accuracy = accuracy_score(y_test, y_pred_class)
    print(f'Accuracy: {accuracy:.4f}')
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred_class))

Accuracy: 0.5222
Confusion Matrix:
[[ 0 86]
 [ 0 94]]
